<a href="https://colab.research.google.com/github/ushifali/final-23/blob/master/Collaborative_Filtering_Restaurants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

# Load user preferences data
user_preferences_df  = pd.read_csv('/content/drive/MyDrive/Filtering_Data/user_preferences.csv', encoding='utf-8')

# Load restaurant data
restaurants_df = pd.read_csv('/content/drive/MyDrive/Filtering_Data/content_filtered_data.csv', encoding='utf-8')

In [8]:

from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import MultiLabelBinarizer


# Define a function to calculate Jaccard similarity between a user's preferences and all restaurants
def calculate_similarity(user_id):
    user_prefs = user_preferences_df[user_preferences_df['user_id'] == user_id].iloc[:, 3:].values
    restaurant_features = restaurants_df.iloc[:, 3:].values

    # Create a MultiLabelBinarizer object to one-hot encode the values in each column as binary features
    mlb = MultiLabelBinarizer()

    # Fit the MultiLabelBinarizer on all the values in the 'Cuisines' column of the restaurants dataframe
    mlb.fit(restaurants_df['Cuisines'].str.split(', '))

    # Transform the 'Cuisines' column of both user preferences and restaurants dataframes into binary features
    user_cuisines = mlb.transform(user_preferences_df[user_preferences_df['user_id'] == user_id]['categories'].str.split(', '))
    restaurant_cuisines = mlb.transform(restaurants_df['Cuisines'].str.split(', '))

    # Calculate Jaccard similarity between the user's preferences and all restaurants
    similarities = pairwise_distances(user_cuisines, restaurant_cuisines, metric='jaccard')

    return 1 - similarities[0]






Top 5 restaurant recommendations for user 1:
['Basant Roving Feast', 'SRKP Chettinad Restaurant', 'Asha Tiffins', 'Chai Kings', 'Khawa Karpo']


/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['Fast-Food', 'Italian', 'Japanese', 'Mediterranean', 'Mexican', 'Thai'] will be ignored
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [13]:
restaurant_names = restaurants_df['Restaurant_Name']

In [20]:
# Example usage: Recommend top 5 restaurants for user 1
user_id = 1
similarities = calculate_similarity(user_id)
restaurant_data = restaurants_df[['Restaurant_Name', 'Cuisines', 'Pricing_for_2', 'Locality', 'Dining_Rating','Distance']]
recommendations = pd.DataFrame({'Restaurant_Name': restaurant_names, 'Similarity': similarities})
recommendations = recommendations.merge(restaurant_data, on='Restaurant_Name').sort_values(by='Similarity', ascending=False).head(5)



/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['Fast-Food', 'Italian', 'Japanese', 'Mediterranean', 'Mexican', 'Thai'] will be ignored
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [21]:
print(recommendations.to_string(index=False))

          Restaurant_Name  Similarity                                                                                      Cuisines  Pricing_for_2                         Locality  Dining_Rating  Distance
      Basant Roving Feast    0.333333                                                    Chinese, American, Fast Food - Quick Bites          300.0                  Gokulam, Mysore           4.30  1.219701
SRKP Chettinad Restaurant    0.250000                                 South Indian, Chinese, Kebab, Indian, Seafood - Casual Dining          200.0            Peelamedu, Coimbatore           4.30  2.048333
             Asha Tiffins    0.222222 South Indian, North Indian, Chinese, Beverages, Desserts ,South Indian, Chinese - Quick Bites          200.0                   HSR, Bangalore           4.45  0.066930
               Chai Kings    0.166667                                                                   Beverages, Fast Food - Café          200.0         Anna Nagar East, Chennai 